In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

data_path = kagglehub.competition_download('***')
edudum_mahalle_listesi_path = kagglehub.dataset_download('edudum/mahalle-listesi') #NVİ mahalle listesi

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:
! pip install -q datasets
! pip install unidecode -q
! pip install rapidfuzz -q
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
import joblib
from pathlib import Path
import matplotlib as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.1 MB/s eta 0:00:00


In [ ]:


# Build file paths
train_path = os.path.join(data_path, "train.csv")
test_path  = os.path.join(data_path, "test.csv")

# Read CSVs
train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print(train_df.head())



In [ ]:
import csv
import re
from unidecode import unidecode

def normalize_address(text):
    # Lowercase and apply unidecode for Turkish characters
    text = unidecode(text.lower())

    # Expand common abbreviations
    text = re.sub(r'\b(mahalle|mah|mhl|mh)\.?\b', 'mahallesi', text)
    text = re.sub(r'\b(cadde|cadd|cad|cd)\.?\b', 'caddesi', text)
    text = re.sub(r'\b(bulv|blv)\.?\b', 'bulvari', text)
    text = re.sub(r'\b(sok|skk|sk)\.?\b', 'sokak', text)
    text = re.sub(r'\b(blk|bl)\.?\b', 'blok', text)
    text = re.sub(r'\b(no|num)\.?\b', 'numara', text)

    # Fix wrong naming
    text = text.replace('sokagi', 'sokak')

    # Remove TR/Turkiye
    text = text.replace('TR', '')
    text = text.replace('Turkiye', '')
    text = text.replace("tr", "")


    # Replace new lines
    text = text.replace('\n', ' ')

    # Replace slashes '/'
    text = re.sub(r'(?<=[A-Za-z])/(?=[A-Za-z])', ' ', text)

    # Remove all punctuation if not between two digits
    text = re.sub(r'(?<!\d)[^\w\s]|[^\w\s](?!\d)', '', text)

    # Remove excess whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# # Read CSV into list of dictionaries
# with open('/kaggle/input/***/train.csv', newline='', encoding='utf-8') as csvfile:
#     reader = csv.DictReader(csvfile)
#     data = [row for row in reader]

# # Example
# for i in range (10):
#     address = data[i]["address"]
#     print(normalize_address(address))


In [ ]:
mahalle_excel_path = os.path.join(edudum_mahalle_listesi_path, "Mahalle_Listesi.xls")
gazetteer_df = pd.read_excel(mahalle_excel_path)

gazetteer_df.columns = ['sira', 'mahalle_adi', 'baglilik_bilgisi']

In [ ]:
# !pip -q install tqdm tenacity
# import os, json, time, math, pandas as pd
# import requests
# from tqdm.auto import tqdm
# from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type


In [ ]:
## demo for understanding performance of GPT's on this task.
# os.environ["OPENROUTER_API_KEY"] = os.environ.get("OPENROUTER_API_KEY", "API_KEY")

# OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
# MODEL = "deepseek/deepseek-chat-v3-0324:free"  # ücretsiz rota

# HEADERS = {
#     "Authorization": f"Bearer {os.environ['OPENROUTER_API_KEY']}",
#     "Content-Type": "application/json",
# }

# SYSTEM_PROMPT = (
#     "Görevin: Türkçe serbest metin adreslerini normalize etmek.\n"
#     "- Yazım hatalarını düzelt (Türkçe karakterleri koru).\n"
#     "- Kısaltmaları genişlet: 'mah'->'Mahallesi', 'cad'->'Caddesi', 'sk'/'sok'->'Sokak', "
#     "'blv'->'Bulvarı', 'no'/'num'->'No'.\n"
#     "- Yapışık yazılanları ayır: 'cadde5'->'Cadde 5', 'no10'->'No 10', 'kat3'->'Kat 3'.\n"
#     "- Anlamı değiştirme, veri kaybetme: tüm sayı, blok/daire/kat/site vs. kalsın.\n"
#     "- İl/ilçe/mahalle adlarını büyük harf uyumlu yaz (sadece özel adların ilk harfi büyük).\n"
#     "- Sadece normalize edilmiş adresi tek satırda döndür; başka açıklama ekleme."
# )

# def build_user_prompt(addr: str) -> str:
#     return f"Adres: {addr}"

# class TransientAPIError(Exception):
#     pass

# @retry(
#     wait=wait_exponential(multiplier=1, min=1, max=60),
#     stop=stop_after_attempt(6),
#     retry=retry_if_exception_type(TransientAPIError),
# )
# def normalize_address_llm(addr: str) -> str:
#     payload = {
#         "model": MODEL,
#         "messages": [
#             {"role": "system", "content": SYSTEM_PROMPT},
#             {"role": "user", "content": build_user_prompt(addr)}
#         ],
#         "temperature": 0,
#         "max_tokens": 256,
#     }
#     r = requests.post(OPENROUTER_URL, headers=HEADERS, json=payload, timeout=60)
#     if r.status_code in (429, 500, 502, 503, 504):
#         # geçici hata → yeniden dene
#         raise TransientAPIError(f"{r.status_code} {r.text[:200]}")
#     r.raise_for_status()
#     data = r.json()
#     out = data["choices"][0]["message"]["content"].strip()
#     # Güvenlik: tek satır ve kontrol
#     return " ".join(out.split())


In [ ]:
# tests = [
#     "cumhurıyet mah.hukumet cad.olcay koç ishani no5 kat2 daire5 fethiye/mugla",
#     "dumlupınar mah 1234 sk no14/5 d blok kat1 d4 izmir karşıyaka",
#     "bahcelievler mh 8.cad 24.sk no:13a site4 blokb daire:25 Ankara/çankaya"
# ]
# for t in tests:
#     print("IN :", t)
#     print("OUT:", normalize_address_llm(t))
#     print("---")


In [ ]:
# downloading zemberek nlp model for normalization and etc.
# !wget -O zemberek-full.jar "https://drive.google.com/uc?export=download&id=1RRuFK43JqcHcthB3fV2IEpPftWoeoHAu"
!gdown --id 1RRuFK43JqcHcthB3fV2IEpPftWoeoHAu -O zemberek-full.jar
!pip install jpype1
!pip install zemberek-python
import gdown

# Download normalization directory
gdown.download_folder('https://drive.google.com/drive/folders/1jNT6BJoEbiLuVbQwBYdVNoibEdzDd2WC?usp=drive_link', quiet=False)

# Download lm.2gram.slm file
gdown.download('https://drive.google.com/uc?export=download&id=1JZG0I8jUS511lFVg0M-QAA4QRqydlCiX', 'lm.2gram.slm', quiet=False)

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1RRuFK43JqcHcthB3fV2IEpPftWoeoHAu
From (redirected): https://drive.google.com/uc?id=1RRuFK43JqcHcthB3fV2IEpPftWoeoHAu&confirm=t&uuid=88d5a625-ce66-4122-a81d-83f4173edc22
To: /content/zemberek-full.jar
100% 31.6M/31.6M [00:00<00:00, 57.7MB/s]


# **Citation:** <br>
***zemberek-nlp*** <br>
https://github.com/ahmetaa/zemberek-nlp/tree/master

In [ ]:
## demo to understand how zemberek works
# %%writefile NormalizeAddress.java
# import zemberek.normalization.TurkishSentenceNormalizer;
# import zemberek.morphology.TurkishMorphology;
# import java.nio.file.Path;

# public class NormalizeAddress {
#     public static void main(String[] args) throws Exception {
#         TurkishMorphology morphology = TurkishMorphology.createWithDefaults();

#         Path root = Path.of("zemberek-data"); // dil modeli dosyaları

#         TurkishSentenceNormalizer normalizer =
#             new TurkishSentenceNormalizer(
#                 morphology,
#                 root.resolve("normalization"),
#                 root.resolve("lm")
#             );

#         String input = "ankra cadd5 no12 dky mah.";
#         String normalized = normalizer.normalize(input);
#         System.out.println("Input   : " + input);
#         System.out.println("Output  : " + normalized);
#     }
# }


Overwriting NormalizeAddress.java


In [ ]:
# !javac -cp zemberek-full.jar NormalizeAddress.java

In [ ]:
# !java -cp .:zemberek-full.jar NormalizeAddress


In [ ]:
# import jpype
# from jpype import JClass

# # Start the JVM
# jpype.startJVM(jpype.getDefaultJVMPath(), '-ea', '-Djava.class.path=path_to_zemberek_full.jar')

# # Load necessary classes
# TurkishSentenceNormalizer = JClass('zemberek.normalization.TurkishSentenceNormalizer')
# lm_path = 'lm.2gram.slm'
# normalizer = TurkishSentenceNormalizer(lm_path)

# # Example sentence
# sentence = 'Bugün hava çok güzel. Hadi dışarı çıkalım.'

# # Normalize the sentence
# normalized_sentence = normalizer.normalize(sentence)
# print(normalized_sentence)


###
> ### IMPLEMENTATION STARTS FROM HERE ON!
###

In [ ]:

! pip install -q datasets
! pip install unidecode -q
! pip install rapidfuzz -q
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
import joblib
from pathlib import Path
import matplotlib as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data_path = kagglehub.competition_download('***')
edudum_mahalle_listesi_path = kagglehub.dataset_download('edudum/mahalle-listesi')

print('Data source import complete.')

# !wget -O zemberek-full.jar "https://drive.google.com/uc?export=download&id=1RRuFK43JqcHcthB3fV2IEpPftWoeoHAu"
!gdown --id 1RRuFK43JqcHcthB3fV2IEpPftWoeoHAu -O zemberek-full.jar
!pip install jpype1
!pip install zemberek-python
import gdown

# Download normalization directory
gdown.download_folder('https://drive.google.com/drive/folders/1jNT6BJoEbiLuVbQwBYdVNoibEdzDd2WC?usp=drive_link', quiet=False)

# Download lm.2gram.slm file
gdown.download('https://drive.google.com/uc?export=download&id=1JZG0I8jUS511lFVg0M-QAA4QRqydlCiX', 'lm.2gram.slm', quiet=False)

In [ ]:
# -----------------------------------------
# PRE-NORMALIZATION (before Zemberek)
# -----------------------------------------
import os
import re
import pandas as pd
from unidecode import unidecode

# -------- Manual pre-normalization --------
def pre_normalize_manual(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = unidecode(text.lower())
    # Common abbreviations
    text = re.sub(r'\b(mahalle|mah|mhl|mh)\b', 'mahallesi', text)
    text = re.sub(r'\b(cadde|cadd|cad|cd)\b', 'caddesi', text)
    text = re.sub(r'\b(sokak|sok|skk|sk)\b', 'sokagi', text)
    text = re.sub(r'\b(bulvari|bulv|blv)\b', 'bulvari', text)
    text = re.sub(r'\b(apartmani|apt|ap)\b', 'apartmani', text)
    text = re.sub(r'\b(numara|no|nu)\b', 'numara', text)
    text = re.sub(r'\b(daire|dai|d)\b', 'daire', text)
    text = re.sub(r'\b(kat|k)\b', 'kat', text)
    text = re.sub(r'\b(blok|blk|bl)\b', 'blok', text)
    text = re.sub(r'\b(tr|turkiye|turkey)\b', '', text)

    # Replace slashes, punctuation etc.
    text = re.sub(r'/', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    PRE_ABBR = [
        (re.compile(r"\bcd\.?\s*no\b", flags=re.IGNORECASE), "cadde no"),
        (re.compile(r"\bcd\.?\b", flags=re.IGNORECASE), "cadde"),
        (re.compile(r"\bsk\.?\s*no\b", flags=re.IGNORECASE), "sokak no"),
        (re.compile(r"\bsk\.?\b", flags=re.IGNORECASE), "sokak"),
        (re.compile(r"\bm[hl]?\.?\b", flags=re.IGNORECASE), "mah"),
        (re.compile(r"\bb(\d+)\b", flags=re.IGNORECASE), r"blok \1"),
        (re.compile(r"\bd(\d+)\b", flags=re.IGNORECASE), r"daire \1"),
    ]

    for pat, rep in PRE_ABBR:
        text = pat.sub(rep, text)
    return text

# -------- Load CSVs --------
train_path = os.path.join(data_path, "train.csv")
test_path  = os.path.join(data_path, "test.csv")

train_df = pd.read_csv(train_path)#.head(50)   # first 50 rows
test_df  = pd.read_csv(test_path)#.head(50)

# ⚠️ adjust this if your address column has another name
ADDRESS_COL = "address"

train_addrs = train_df[ADDRESS_COL].dropna().astype(str).tolist()
test_addrs  = test_df[ADDRESS_COL].dropna().astype(str).tolist()

# -------- Apply manual pre-normalization --------
all_addrs = [pre_normalize_manual(addr) for addr in (train_addrs + test_addrs)]



# -------- Save to file for Java --------
clean_addresses = []
with open("to_normalize.txt", "w", encoding="utf-8") as f:
    for addr in all_addrs:
        if addr and addr.strip():   # skip empty or whitespace-only
            f.write(addr.strip() + "\n")
            clean_addresses.append(addr.strip())

print(f"Saved {len(clean_addresses)} clean addresses to to_normalize.txt")



Saved 1065475 clean addresses to to_normalize.txt


In [ ]:
%%writefile NormalizeFullSentence.java
import zemberek.morphology.TurkishMorphology;
import zemberek.normalization.TurkishSentenceNormalizer;
import java.nio.file.*;
import java.io.*;
import java.util.*;

public class NormalizeFullSentence {
    public static void main(String[] args) throws Exception {

        // Create default Turkish morphology
        TurkishMorphology morphology = TurkishMorphology.createWithDefaults();

        // Set paths to normalization resources
        Path zemberekDataRoot = Paths.get("/content/"); // adjust to your root folder
        Path lookupRoot = zemberekDataRoot.resolve("normalization");
        Path lmPath = zemberekDataRoot.resolve("lm.2gram.slm");

        // Initialize the TurkishSentenceNormalizer
        TurkishSentenceNormalizer normalizer =
            new TurkishSentenceNormalizer(morphology, lookupRoot, lmPath);

        // Read lines from file
        List<String> inputs = Files.readAllLines(Paths.get("to_normalize.txt"));

        // Normalize and print
        for (String sentence : inputs) {
            try {
                String normalized = normalizer.normalize(sentence);
                System.out.println(normalized);
            } catch (Exception e) {
                System.out.println(sentence); // fallback: print original
            }
        }
    }
}


Writing NormalizeFullSentence.java


In [ ]:
!javac -cp zemberek-full.jar NormalizeFullSentence.java


In [ ]:
!java -cp .:zemberek-full.jar NormalizeFullSentence > zemberek_output.txt


I|08:43:21.949|Root lexicon created in 2179 ms.                                                                    | DictionarySerializer#getDictionaryItems
I|08:43:21.953|Dictionary generated in 2458 ms                                                                     | RootLexicon#defaultBinaryLexicon
I|08:43:23.551|Initialized in 4538 ms.                                                                             | TurkishMorphology#createWithDefaults
I|08:43:25.402|Language model = Order : 2
1 Grams: Count= 300003
2 Grams: Count= 19499916  Fingerprint Bits= 24  Probabilty Bits= 8  Back-off bits= 0
Log Base              : 2.72
Unigram Weight        : 1.00
Using Stupid Back-off?: No
Unknown Back-off N-gram penalty: 0.00
| TurkishSentenceNormalizer#<init>


In [ ]:
!wc -l zemberek_output.txt


1065475 zemberek_output.txt


In [ ]:
# !java -cp .:zemberek-full.jar NormalizeFullSentence > zemberek_output.txt

!java -jar zemberek-full.jar train_addresses.txt > zemberek_train_output.txt
!java -jar zemberek-full.jar test_addresses.txt > zemberek_test_output.txt


In [ ]:
!wc -l train_addresses.txt
!wc -l zemberek_train_output.txt


wc: train_addresses.txt: No such file or directory
73 zemberek_train_output.txt


In [ ]:
# import os
# import pandas as pd
# from rapidfuzz import process

# # --- Load your official gazetteer ---
# mahalle_excel_path = os.path.join(edudum_mahalle_listesi_path, "Mahalle_Listesi.xls")
# gazetteer_df = pd.read_excel(mahalle_excel_path)

# # Rename properly to lowercase/underscore versions
# gazetteer_df = gazetteer_df.rename(columns={
#     "SIRA": "sira",
#     "MAHALLE ADI ": "mahalle_adi",
#     "MAHALLENİN BAĞLILIK BİLGİSİ": "baglilik_bilgisi"
# })

# # Split province, district, central_unit
# gazetteer_df[['province', 'district', 'central_unit']] = (
#     gazetteer_df['baglilik_bilgisi'].str.split(' -> ', expand=True, n=2)
# )

# # Drop rows with missing essentials
# gazetteer_df = gazetteer_df.dropna(subset=['province', 'district', 'mahalle_adi'])

# # Build gazetteer lists
# provinces = gazetteer_df['province'].dropna().unique().tolist()
# districts = gazetteer_df['district'].dropna().unique().tolist()
# mahalleler = gazetteer_df['mahalle_adi'].dropna().unique().tolist()

# gazetteer = {
#     'provinces': provinces,
#     'districts': districts,
#     'mahalleler': mahalleler,
#     'province_to_districts': gazetteer_df.groupby('province')['district'].apply(set).to_dict()
# }

# # --- Read Zemberek outputs ---
# with open("zemberek_output.txt", "r") as f:
#     normalized_addresses = [line.strip() for line in f if line.strip()]

# # --- Fuzzy matching helper ---
# def match_with_gazetteer(text, gazetteer):
#     province = process.extractOne(text, gazetteer['provinces'], score_cutoff=80)
#     district = process.extractOne(text, gazetteer['districts'], score_cutoff=80)
#     mahalle = process.extractOne(text, gazetteer['mahalleler'], score_cutoff=80)
#     return {
#         "province": province[0] if province else None,
#         "district": district[0] if district else None,
#         "mahalle": mahalle[0] if mahalle else None
#     }

# # --- Process all addresses ---
# results = []
# for addr in normalized_addresses:
#     match = match_with_gazetteer(addr, gazetteer)
#     results.append({
#         "normalized_zemberek": addr,
#         "matched_province": match['province'],
#         "matched_district": match['district'],
#         "matched_mahalle": match['mahalle']
#     })

# results_df = pd.DataFrame(results)
# results_df.head(10)


In [ ]:
# import os, re
# import pandas as pd
# from rapidfuzz import process, fuzz
# from unidecode import unidecode

# # ---------------------------
# # Helpers
# # ---------------------------
# def norm(s: str) -> str:
#     return unidecode(str(s)).lower().strip()

# def has_word(a_norm: str, token_norm: str) -> bool:
#     # kelime sınırlarıyla alt dize araması
#     return re.search(rf'(?<!\w){re.escape(token_norm)}(?!\w)', a_norm) is not None

# def extract_mahalle_phrase(addr_norm: str) -> str | None:
#     # "... <isim> mah." veya "... <isim> mahallesi"
#     m = re.search(r'([a-z0-9çğıöşü\-\s]{2,60})\s+m(?:ah(?:\.|allesi)?)\b', addr_norm)
#     if not m:
#         return None
#     phrase = m.group(1).strip()
#     toks = [t for t in phrase.split() if t and not t.isdigit()]
#     if not toks:
#         return None
#     # Mahalle adı genelde sonda; son 1–3 token'ı odak al
#     focus = " ".join(toks[-3:])
#     return focus

# def best_match(query: str, candidates: list[str], cutoff=85, scorer=fuzz.token_set_ratio):
#     if not candidates:
#         return None, 0
#     hit = process.extractOne(query, candidates, scorer=scorer, score_cutoff=cutoff)
#     return (hit[0], hit[1]) if hit else (None, 0)

# # ---------------------------
# # Load Gazetteer
# # ---------------------------
# mahalle_excel_path = os.path.join(edudum_mahalle_listesi_path, "Mahalle_Listesi.xls")
# raw = pd.read_excel(mahalle_excel_path)

# # Kolon adlarını güvenli biçimde eşle
# name_map = {}
# for c in raw.columns:
#     lc = unidecode(c).lower().strip()
#     if lc.startswith("sira"):
#         name_map[c] = "sira"
#     elif "mahalle" in lc and "ad" in lc:
#         name_map[c] = "mahalle_adi"
#     elif "baglilik" in lc:
#         name_map[c] = "baglilik_bilgisi"

# gazetteer_df = raw.rename(columns=name_map)[["sira", "mahalle_adi", "baglilik_bilgisi"]].copy()
# gazetteer_df[["province", "district", "central_unit"]] = (
#     gazetteer_df["baglilik_bilgisi"].astype(str).str.split(" -> ", expand=True, n=2)
# )
# gazetteer_df = gazetteer_df.dropna(subset=["province", "district", "mahalle_adi"])

# # Normalize columns for matching
# gazetteer_df["prov_norm"] = gazetteer_df["province"].map(norm)
# gazetteer_df["dist_norm"] = gazetteer_df["district"].map(norm)
# gazetteer_df["mah_norm"]  = gazetteer_df["mahalle_adi"].map(norm)

# # Unique + mapping (norm -> orijinal)
# prov_map = gazetteer_df.drop_duplicates("prov_norm").set_index("prov_norm")["province"].to_dict()
# dist_map = gazetteer_df.drop_duplicates("dist_norm").set_index("dist_norm")["district"].to_dict()
# mah_map  = gazetteer_df.drop_duplicates("mah_norm").set_index("mah_norm")["mahalle_adi"].to_dict()

# prov_list_norm = list(prov_map.keys())
# dist_list_norm = list(dist_map.keys())
# mah_list_norm  = list(mah_map.keys())

# # İl -> ilçeler (normalize)
# dist_by_prov_norm = (
#     gazetteer_df.groupby("prov_norm")["dist_norm"]
#     .apply(lambda s: set(s.dropna().unique().tolist()))
#     .to_dict()
# )

# # ---------------------------
# # Read Zemberek outputs
# # ---------------------------
# with open("zemberek_output.txt", "r", encoding="utf-8") as f:
#     normalized_addresses = [line.strip() for line in f if line.strip()]

# # ---------------------------
# # Matching
# # ---------------------------
# def match_with_gazetteer(addr_text: str):
#     a_norm = norm(addr_text)
#     # parçalar: / , -
#     parts = re.split(r"[,/|\-]", a_norm)
#     parts = [" ".join([t for t in p.split() if t]) for p in parts]

#     # ---- Province (direct hit first)
#     prov_hits = []
#     for p_norm in prov_list_norm:
#         if any(has_word(part, p_norm) for part in parts):
#             prov_hits.append(p_norm)
#     if prov_hits:
#         prov_norm = max(prov_hits, key=len)  # en uzun adı seç (daha ayırt edici)
#     else:
#         prov_norm, _ = best_match(a_norm, prov_list_norm, cutoff=87)

#     # ---- District (restrict by province if found)
#     if prov_norm and prov_norm in dist_by_prov_norm:
#         dist_candidates = list(dist_by_prov_norm[prov_norm])
#     else:
#         dist_candidates = dist_list_norm
#     dist_norm, _ = best_match(a_norm, dist_candidates, cutoff=86)

#     # ---- Mahalle (context-aware around 'mah')
#     mah_phrase = extract_mahalle_phrase(a_norm)
#     if mah_phrase:
#         mah_norm, _ = best_match(mah_phrase, mah_list_norm, cutoff=85)
#     else:
#         # bağlam yoksa daha sıkı eşik kullan
#         mah_norm, _ = best_match(a_norm, mah_list_norm, cutoff=92)

#     return {
#         "province": prov_map.get(prov_norm),
#         "district": dist_map.get(dist_norm),
#         "mahalle":  mah_map.get(mah_norm),
#     }

# # ---------------------------
# # Run
# # ---------------------------
# rows = []
# for addr in normalized_addresses:
#     m = match_with_gazetteer(addr)
#     rows.append({
#         "normalized_zemberek": addr,
#         "matched_province": m["province"],
#         "matched_district": m["district"],
#         "matched_mahalle":  m["mahalle"],
#     })

# results_df = pd.DataFrame(rows)
# print(results_df.head(10))


In [ ]:
results_df.iloc[7]["normalized_zemberek"]

'bahçelivlr mh 7. cad 23. şano : 12a site3blokb daire : 21 Çankaya'

In [ ]:
# ============================================
# Address normalization + segmentation + gazetteer matching
# (Zemberek çıktısına ek kural seti)
# ============================================

import os, re
import pandas as pd
from unidecode import unidecode
from rapidfuzz import process, fuzz

# ---------------------------
# 0) PATHS
# ---------------------------
# Zemberek çıktısı (satır başına bir normalize edilmiş adres)
ZEMBEREK_OUTPUT = "zemberek_output.txt"

# Mahalle_Listesi.xls'in olduğu klasör
# (örn: edudum_mahalle_listesi_path = "/content/.cache/..../versions/1")
mahalle_excel_path = os.path.join(edudum_mahalle_listesi_path, "Mahalle_Listesi.xls")

# ---------------------------
# 1) NORMALIZATION HELPERS
# ---------------------------

# Küçük harf + TR diakritiklerini KORU (sınırlı ASCII'ye çevirmiyoruz).
def lower_strip(s: str) -> str:
    return s.casefold().strip()

# Hız için regex'leri derleyelim
SPACES = re.compile(r"\s+")
PUNCT_SPACING = re.compile(r"\s*([:/\-.,])\s*")
DOT_NUMBER = re.compile(r"\b(\d+)\.(?=\s*(cad|cadd|cadde|sk|sok|sokak)\b)")  # 7.cad -> 7. cad
SLASH_PAIR = re.compile(r"\b(\d+)\s*/\s*([0-9a-zA-Z]+)\b")                     # 10/4 -> no 10 daire 4 (aşağıda bağlama göre)

# Etiket (mah, cad, sk vs.) genişletmeleri — varyasyonları tek tipe çekiyoruz.
ABBR_PATTERNS = [
    (re.compile(r"\b(mahalle|mah\.?|mhl|mh)\b"),          "mahallesi"),
    (re.compile(r"\b(cadde|cadd|cad\.?|cd)\b"),           "cadde"),
    (re.compile(r"\b(bulvar[ıi]?|bulv|blv)\b"),           "bulvar"),
    (re.compile(r"\b(sokak|sok\.?|skk|sk)\b"),            "sokak"),
    (re.compile(r"\b(apartman[ıi]?|apartmani|apt|ap)\b"), "apartman"),
    (re.compile(r"\b(site(si)?)\b"),                      "sitesi"),
    (re.compile(r"\b(no|num|numara)\b"),                  "no"),
    (re.compile(r"\b(daire|dai)\b"),                      "daire"),
    (re.compile(r"\b(kat)\b"),                            "kat"),
    (re.compile(r"\b(blok|blk|bl)\b"),                    "blok"),
]

# “b1” -> “blok 1”, “d5” -> “daire 5” gibi kompakt yazımlar.
COMPACT_NUM = [
    (re.compile(r"(?<!\w)b\s*(\d+)\b"), "blok \\1"),
    # Daire: sadece sayı takip ediyorsa (örn: d:5, d5). "d blok"u bozmayalım diye iki kural:
    (re.compile(r"\bd\s*[:\-]?\s*(\d+)\b(?!\s*blok)"), "daire \\1"),
]

# “no 10/4” ya da “10/4” -> “no 10 daire 4”
def fix_no_slash_pairs(text: str) -> str:
    def _repl(m):
        left, right = m.group(1), m.group(2)
        # Eğer hemen öncesinde “no” geçiyorsa sadece “daire” ekle
        before = text[:m.start()]
        if re.search(r"no\s*$", before):
            return f"{left} daire {right}"
        return f"no {left} daire {right}"
    return SLASH_PAIR.sub(_repl, text)

# “no:12”, “kat:3”, “d:5” gibi yazımları standardize et -> “no 12”, “kat 3”, “daire 5”
LABEL_COLON = [
    (re.compile(r"\bno\s*[:=]\s*"), "no "),
    (re.compile(r"\bkat\s*[:=]\s*"), "kat "),
    (re.compile(r"\bdaire\s*[:=]\s*"), "daire "),
    (re.compile(r"\bblok\s*[:=]\s*"), "blok "),
]

# “kat3” -> “kat 3”, “daire21” -> “daire 21”, “blokb” -> “blok b”
ATTACH_FIX = [
    (re.compile(r"\b(kat)(\d+)\b"), r"\1 \2"),
    (re.compile(r"\b(daire)(\d+)\b"), r"\1 \2"),
    (re.compile(r"\b(blok)([a-z0-9]+)\b"), r"\1 \2"),
]

# Bazı gürültüleri temizle
NOISE = [
    (re.compile(r"\btr\b|\bturkiye\b|\bturkey\b"), " "),
    (re.compile(r"(?<=\d)[,](?=\d)"), "."),  # 1,234 -> 1.234 (nokta veya boşluk isteniyorsa değiştir)
]

def normalize_manual(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = lower_strip(text)

    # Noktalama çevresi boşluklar ("/", ":", ".", ",", "-")
    t = PUNCT_SPACING.sub(r" \1 ", t)
    # 7.cad -> 7. cad
    t = DOT_NUMBER.sub(r"\1. ", t)

    # Kısaltmaları genişlet
    for pat, rep in ABBR_PATTERNS:
        t = pat.sub(rep, t)

    # Kompakt kısaltmalar
    for pat, rep in COMPACT_NUM:
        t = pat.sub(rep, t)

    # no:12 -> no 12 vb.
    for pat, rep in LABEL_COLON:
        t = pat.sub(rep, t)

    # 10/4 -> no 10 daire 4 (bağlama duyarlı)
    t = fix_no_slash_pairs(t)

    # kat3 -> kat 3, daire21 -> daire 21, blokb -> blok b
    for pat, rep in ATTACH_FIX:
        t = pat.sub(rep, t)

    # Gürültüler
    for pat, rep in NOISE:
        t = pat.sub(rep, t)

    # Çoklu boşluk
    t = SPACES.sub(" ", t).strip()
    return t


# ---------------------------
# 2) SEGMENTATION (alan çıkarımı)
# ---------------------------
# Daha istikrarlı sonuç için “etiket + değer” yakala.
R_MAH = re.compile(r"(?P<name>[a-z0-9\-\s]{2,60})\s+mahallesi\b")
R_CAD = re.compile(r"(?P<name>[0-9a-z\-\.\s]{1,60})\s+cadde\b")
R_SOK = re.compile(r"(?P<name>[0-9a-z\-\.\s]{1,60})\s+sokak\b")
R_BUL = re.compile(r"(?P<name>[0-9a-z\-\.\s]{1,60})\s+bulvar\b")
R_SITE = re.compile(r"\b(?P<name>[0-9a-z\-\.\s]{2,80})\s+sitesi\b")
R_NO = re.compile(r"\bno\s+(?P<val>[0-9a-z]+)\b")
R_KAT = re.compile(r"\bkat\s+(?P<val>[0-9a-z\-]+)\b")
R_DAI = re.compile(r"\bdaire\s+(?P<val>[0-9a-z\-]+)\b")
R_BLOK = re.compile(r"\bblok\s+(?P<val>[0-9a-z\-]+)\b")

def _tail_tokens(s, k=3):
    toks = [tok for tok in s.split() if tok and not tok.isdigit()]
    return " ".join(toks[-k:]) if toks else None

def segment_address(addr: str) -> dict:
    a = addr

    def pick(m, field, k=3):
        if not m:
            return None
        name = m.group("name").strip()
        return _tail_tokens(name, k=k)

    seg = {
        "mah": pick(R_MAH.search(a), "mah"),
        "cadde": pick(R_CAD.search(a), "cadde"),
        "sokak": pick(R_SOK.search(a), "sokak"),
        "bulvar": pick(R_BUL.search(a), "bulvar"),
        "site": pick(R_SITE.search(a), "site", k=4),
        "no": (R_NO.search(a).group("val") if R_NO.search(a) else None),
        "kat": (R_KAT.search(a).group("val") if R_KAT.search(a) else None),
        "daire": (R_DAI.search(a).group("val") if R_DAI.search(a) else None),
        "blok": (R_BLOK.search(a).group("val") if R_BLOK.search(a) else None),
    }
    return seg


# ---------------------------
# 3) GAZETTEER YÜKLE + NORMALİZE
# ---------------------------
raw = pd.read_excel(mahalle_excel_path)

# Kolon isimleri değişebilir; güvenli yeniden adlandırma:
name_map = {}
for c in raw.columns:
    lc = unidecode(str(c)).lower().strip()
    if lc.startswith("sira"):
        name_map[c] = "sira"
    elif "mahalle" in lc and "ad" in lc:
        name_map[c] = "mahalle_adi"
    elif "baglilik" in lc:
        name_map[c] = "baglilik_bilgisi"

gazetteer_df = raw.rename(columns=name_map)[["sira", "mahalle_adi", "baglilik_bilgisi"]].copy()
gazetteer_df[["province", "district", "central_unit"]] = (
    gazetteer_df["baglilik_bilgisi"].astype(str).str.split(" -> ", expand=True, n=2)
)
gazetteer_df = gazetteer_df.dropna(subset=["province", "district", "mahalle_adi"])

# Normalizasyon (eşleşme için)
gazetteer_df["prov_norm"] = gazetteer_df["province"].map(lambda x: unidecode(str(x)).lower().strip())
gazetteer_df["dist_norm"] = gazetteer_df["district"].map(lambda x: unidecode(str(x)).lower().strip())
gazetteer_df["mah_norm"]  = gazetteer_df["mahalle_adi"].map(lambda x: unidecode(str(x)).lower().strip())

prov_map = gazetteer_df.drop_duplicates("prov_norm").set_index("prov_norm")["province"].to_dict()
dist_map = gazetteer_df.drop_duplicates("dist_norm").set_index("dist_norm")["district"].to_dict()
mah_map  = gazetteer_df.drop_duplicates("mah_norm").set_index("mah_norm")["mahalle_adi"].to_dict()

prov_list_norm = list(prov_map.keys())
dist_list_norm = list(dist_map.keys())
mah_list_norm  = list(mah_map.keys())

# il -> ilçe kısıtı
dist_by_prov_norm = (
    gazetteer_df.groupby("prov_norm")["dist_norm"]
    .apply(lambda s: set(s.dropna().unique().tolist()))
    .to_dict()
)

def best_match(query: str, candidates: list[str], cutoff=86, scorer=fuzz.token_set_ratio):
    if not candidates:
        return None, 0
    hit = process.extractOne(query, candidates, scorer=scorer, score_cutoff=cutoff)
    return (hit[0], hit[1]) if hit else (None, 0)

# mahalle bağlamsal: “... X mahallesi” içinde geçen kısım
def extract_mah_focus(addr_norm: str) -> str | None:
    m = re.search(r"([a-z0-9\-\s]{2,60})\s+mahallesi\b", addr_norm)
    if not m:
        return None
    return _tail_tokens(m.group(1), k=3)

def match_with_gazetteer(addr_text_norm: str, seg: dict):
    # Eşleşmeyi ASCII normal üzerinde yapıyoruz
    a_norm = unidecode(addr_text_norm).lower().strip()
    parts = re.split(r"[,/|\-]", a_norm)
    parts = [" ".join([t for t in p.split() if t]) for p in parts]

    # İl: önce doğrudan parçalarda, yoksa fuzzy
    prov_hits = [p for p in prov_list_norm if any(re.search(rf"(?<!\w){re.escape(p)}(?!\w)", part) for part in parts)]
    if prov_hits:
        prov_norm = max(prov_hits, key=len)
    else:
        prov_norm, _ = best_match(a_norm, prov_list_norm, cutoff=87)

    # İlçe: il'e bağlı havuzda dene, yoksa genelde
    dist_candidates = list(dist_by_prov_norm.get(prov_norm, [])) or dist_list_norm
    dist_norm, _ = best_match(a_norm, dist_candidates, cutoff=86)

    # Mahalle: önce regex bağlamı, yoksa tüm havuz
    focus = extract_mah_focus(a_norm) or seg.get("mah")
    if focus:
        mah_norm, _ = best_match(focus, mah_list_norm, cutoff=85)
    else:
        mah_norm, _ = best_match(a_norm, mah_list_norm, cutoff=92)

    return {
        "province": prov_map.get(prov_norm),
        "district": dist_map.get(dist_norm),
        "mahalle_official": mah_map.get(mah_norm),
    }

# ---------------------------
# 4) PIPELINE: Zemberek -> Regex Normalization -> Segmentation -> Gazetteer
# ---------------------------
with open(ZEMBEREK_OUTPUT, "r", encoding="utf-8") as f:
    z_out = [line.strip() for line in f if line.strip()]

from tqdm import tqdm

rows = []
for addr in tqdm(z_out, desc="Processing addresses"):
    norm_addr = normalize_manual(addr)
    seg = segment_address(norm_addr)
    g = match_with_gazetteer(norm_addr, seg)
    rows.append({
        "zemberek_out": addr,
        "norm_rule_based": norm_addr,
        "seg_mah": seg["mah"],
        "seg_cadde": seg["cadde"],
        "seg_sokak": seg["sokak"],
        "seg_bulvar": seg["bulvar"],
        "seg_site": seg["site"],
        "seg_blok": seg["blok"],
        "seg_no": seg["no"],
        "seg_kat": seg["kat"],
        "seg_daire": seg["daire"],
        "match_province": g["province"],
        "match_district": g["district"],
        "match_mahalle_official": g["mahalle_official"],
    })

results_df = pd.DataFrame(rows)

# # İsteğe bağlı: kanonik adres üretme (alanları sıralı bir formata toplama)
# def canonicalize(row):
#     chunks = []
#     if row.get("seg_mah"):    chunks += [f"{row['seg_mah']} mahallesi"]
#     if row.get("seg_cadde"):  chunks += [f"{row['seg_cadde']} cadde"]
#     if row.get("seg_sokak"):  chunks += [f"{row['seg_sokak']} sokak"]
#     if row.get("seg_bulvar"): chunks += [f"{row['seg_bulvar']} bulvar"]
#     if row.get("seg_site"):   chunks += [f"{row['seg_site']} sitesi"]
#     if row.get("seg_blok"):   chunks += [f"blok {row['seg_blok']}"]
#     if row.get("seg_no"):     chunks += [f"no {row['seg_no']}"]
#     if row.get("seg_kat"):    chunks += [f"kat {row['seg_kat']}"]
#     if row.get("seg_daire"):  chunks += [f"daire {row['seg_daire']}"]
#     # Sonuna idari birimler
#     tail = [row.get("match_district"), row.get("match_province")]
#     tail = [t for t in tail if isinstance(t, str) and t]
#     if tail:
#         chunks.append(" / ".join(tail))
#     return " ".join(chunks).strip()

# results_df["canonical_address"] = results_df.apply(canonicalize, axis=1)

print(results_df.head(10))


In [ ]:
# ============================================
# Address normalization + segmentation + gazetteer matching
# (Uses Zemberek output directly; optional manual normalization for comparison)
# ============================================

import os, re
import pandas as pd
from unidecode import unidecode
from rapidfuzz import process, fuzz
from tqdm import tqdm

# ---------------------------
# 0) PATHS
# ---------------------------
ZEMBEREK_OUTPUT = "zemberek_output.txt"
mahalle_excel_path = os.path.join(edudum_mahalle_listesi_path, "Mahalle_Listesi.xls")

# ---------------------------
# 1) SEGMENTATION HELPERS
# ---------------------------
R_MAH = re.compile(r"(?P<name>[a-z0-9\-\s]{2,60})\s+mahallesi\b")
R_CAD = re.compile(r"(?P<name>[0-9a-z\-\.\s]{1,60})\s+cadde\b")
R_SOK = re.compile(r"(?P<name>[0-9a-z\-\.\s]{1,60})\s+sokak\b")
R_BUL = re.compile(r"(?P<name>[0-9a-z\-\.\s]{1,60})\s+bulvar\b")
R_SITE = re.compile(r"\b(?P<name>[0-9a-z\-\.\s]{2,80})\s+sitesi\b")
R_NO = re.compile(r"\bno\s+(?P<val>[0-9a-z]+)\b")
R_KAT = re.compile(r"\bkat\s+(?P<val>[0-9a-z\-]+)\b")
R_DAI = re.compile(r"\bdaire\s+(?P<val>[0-9a-z\-]+)\b")
R_BLOK = re.compile(r"\bblok\s+(?P<val>[0-9a-z\-]+)\b")

def _tail_tokens(s, k=3):
    toks = [tok for tok in s.split() if tok and not tok.isdigit()]
    return " ".join(toks[-k:]) if toks else None

def segment_address(addr: str) -> dict:
    a = addr
    def pick(m, k=3):
        if not m: return None
        return _tail_tokens(m.group("name").strip(), k=k)

    return {
        "mah": pick(R_MAH.search(a)),
        "cadde": pick(R_CAD.search(a)),
        "sokak": pick(R_SOK.search(a)),
        "bulvar": pick(R_BUL.search(a)),
        "site": pick(R_SITE.search(a), k=4),
        "no": (R_NO.search(a).group("val") if R_NO.search(a) else None),
        "kat": (R_KAT.search(a).group("val") if R_KAT.search(a) else None),
        "daire": (R_DAI.search(a).group("val") if R_DAI.search(a) else None),
        "blok": (R_BLOK.search(a).group("val") if R_BLOK.search(a) else None),
    }

# ---------------------------
# 2) GAZETTEER
# ---------------------------
raw = pd.read_excel(mahalle_excel_path)
name_map = {}
for c in raw.columns:
    lc = unidecode(str(c)).lower().strip()
    if lc.startswith("sira"): name_map[c] = "sira"
    elif "mahalle" in lc and "ad" in lc: name_map[c] = "mahalle_adi"
    elif "baglilik" in lc: name_map[c] = "baglilik_bilgisi"

gazetteer_df = raw.rename(columns=name_map)[["sira", "mahalle_adi", "baglilik_bilgisi"]].copy()
gazetteer_df[["province", "district", "central_unit"]] = (
    gazetteer_df["baglilik_bilgisi"].astype(str).str.split(" -> ", expand=True, n=2)
)
gazetteer_df = gazetteer_df.dropna(subset=["province", "district", "mahalle_adi"])

gazetteer_df["prov_norm"] = gazetteer_df["province"].map(lambda x: unidecode(str(x)).lower().strip())
gazetteer_df["dist_norm"] = gazetteer_df["district"].map(lambda x: unidecode(str(x)).lower().strip())
gazetteer_df["mah_norm"]  = gazetteer_df["mahalle_adi"].map(lambda x: unidecode(str(x)).lower().strip())

prov_map = gazetteer_df.drop_duplicates("prov_norm").set_index("prov_norm")["province"].to_dict()
dist_map = gazetteer_df.drop_duplicates("dist_norm").set_index("dist_norm")["district"].to_dict()
mah_map  = gazetteer_df.drop_duplicates("mah_norm").set_index("mah_norm")["mahalle_adi"].to_dict()

prov_list_norm = list(prov_map.keys())
dist_list_norm = list(dist_map.keys())
mah_list_norm  = list(mah_map.keys())

dist_by_prov_norm = (
    gazetteer_df.groupby("prov_norm")["dist_norm"]
    .apply(lambda s: set(s.dropna().unique().tolist()))
    .to_dict()
)

def best_match(query: str, candidates: list[str], cutoff=86, scorer=fuzz.token_set_ratio):
    if not candidates:
        return None, 0
    hit = process.extractOne(query, candidates, scorer=scorer, score_cutoff=cutoff)
    return (hit[0], hit[1]) if hit else (None, 0)

def extract_mah_focus(addr_norm: str) -> str | None:
    m = re.search(r"([a-z0-9\-\s]{2,60})\s+mahallesi\b", addr_norm)
    return _tail_tokens(m.group(1), k=3) if m else None

def match_with_gazetteer(addr_text_norm: str, seg: dict):
    a_norm = unidecode(addr_text_norm).lower().strip()
    parts = re.split(r"[,/|\-]", a_norm)
    parts = [" ".join([t for t in p.split() if t]) for p in parts]

    prov_hits = [p for p in prov_list_norm if any(re.search(rf"(?<!\w){re.escape(p)}(?!\w)", part) for part in parts)]
    if prov_hits:
        prov_norm = max(prov_hits, key=len)
    else:
        prov_norm, _ = best_match(a_norm, prov_list_norm, cutoff=87)

    dist_candidates = list(dist_by_prov_norm.get(prov_norm, [])) or dist_list_norm
    dist_norm, _ = best_match(a_norm, dist_candidates, cutoff=86)

    focus = extract_mah_focus(a_norm) or seg.get("mah")
    if focus:
        mah_norm, _ = best_match(focus, mah_list_norm, cutoff=85)
    else:
        mah_norm, _ = best_match(a_norm, mah_list_norm, cutoff=92)

    return {
        "province": prov_map.get(prov_norm),
        "district": dist_map.get(dist_norm),
        "mahalle_official": mah_map.get(mah_norm),
    }

# ---------------------------
# 3) PIPELINE: Zemberek → Segmentation → Gazetteer
# ---------------------------
with open(ZEMBEREK_OUTPUT, "r", encoding="utf-8") as f:
    z_out = [line.strip() for line in f if line.strip()]

rows = []
for addr in tqdm(z_out, desc="Processing addresses"):
    seg = segment_address(addr)           # directly on zemberek_out
    g = match_with_gazetteer(addr, seg)
    rows.append({
        "zemberek_out": addr,
        # "norm_rule_based": normalize_manual(addr),  # <– enable only if you want comparison
        "seg_mah": seg["mah"],
        "seg_cadde": seg["cadde"],
        "seg_sokak": seg["sokak"],
        "seg_bulvar": seg["bulvar"],
        "seg_site": seg["site"],
        "seg_blok": seg["blok"],
        "seg_no": seg["no"],
        "seg_kat": seg["kat"],
        "seg_daire": seg["daire"],
        "match_province": g["province"],
        "match_district": g["district"],
        "match_mahalle_official": g["mahalle_official"],
    })

results_df = pd.DataFrame(rows)
print(results_df.head(10))


Processing addresses: 100%|██████████| 1065475/1065475 [1:38:58<00:00, 179.42it/s]


                                        zemberek_out      seg_mah seg_cadde  \
0  akarca mahallesi adnan menderes caddesi 864 so...       akarca      None   
1  cumhuriyet mahallesi hükümet caddesi sivriler ...   cumhuriyet      None   
2  ismet inönü mahallesi 2001 sokağı numara 2 çeş...         None      None   
3  gazeteci hasan tahsin caddesi numara 10 3 gize...         None      None   
4  bitez mahallesi adnan menderes caddesi gündönü...        bitez      None   
5  Derebaşı mahallesi 6100 sokağı numara 10 kat 7...         None      None   
6  dikili güzelselde sitesi müsellim altı numara ...         None      None   
7  yeni sanayi mahallesi 515 sokağı numara 53 c3 ...  yeni sanayi      None   
8  pazaryeri mahallesi 417 sokağı numara 6 4 feth...    pazaryeri      None   
9  gümüşlük mahallesi 5215 sokağı daire blok dış ...         None      None   

  seg_sokak seg_bulvar seg_site seg_blok seg_no seg_kat seg_daire  \
0      None       None     None     None   None       2      

In [ ]:
# ============================================
# 5) TRAIN / TEST sonuçlarını ayır
# ============================================

n_train = len(train_df)
train_rows = results_df.iloc[:n_train].reset_index(drop=True)
test_rows  = results_df.iloc[n_train:].reset_index(drop=True)

# Train sonuçlarına label ekle
train_df_results = pd.concat([train_df.reset_index(drop=True), train_rows], axis=1)
test_df_results  = pd.concat([test_df.reset_index(drop=True),  test_rows], axis=1)

print("Train results with labels:")
print(train_df_results.head(10))

print("\nTest results (no labels):")
print(test_df_results.head(10))


In [ ]:
results_df.shape



In [ ]:
train_df_results.shape

(848237, 16)

In [ ]:
test_df_results.shape

(217241, 16)